# 12 Convolution Layer

In [1]:
import tensorflow as tf
import numpy as np
import time
import os
import re

from PIL import Image as PI

# from tensorflow.examples.tutorials.mnist import input_data
# mnist = input_data.read_data_sets("./mnist/data/", one_hot=True)

    LABEL
        0 -> NORMAL (3,319)
        1 -> PNEUMOTHORAX (2,084)
        2 -> PNEUMONIA (4,277)

# hyper parameter

In [2]:
training_epochs = 500
batch_size = 20
image_resize = 128
image_width = image_resize
image_height = image_resize
initial_rate = 1e-5
label_len = 3

home_dir = "D:/Backup_data/jongkeun"

image_file = "/dir_images/190819_newTraining/"
test_image_file = "/dir_images/190309_new_test/"

label_file = "/csv/190819_newTraining.csv"
test_label_file = "/csv/190309_new_test.csv"

# label, test_label

In [3]:
label_dir = home_dir + label_file
test_label_dir = home_dir + test_label_file

# label_dir = os.getcwd() + "/csv/181126.csv"
# test_label_dir = os.getcwd() + "/csv/181126_test.csv"

f_label = open(label_dir, 'r')
f_test_label = open(test_label_dir, 'r')

label_data = f_label.readlines()
test_label_data = f_test_label.readlines()

f_label.close()
f_test_label.close()

label_array = np.array(np.reshape(label_data, [-1, label_len]), dtype=np.int32)
test_label_array = np.array(np.reshape(test_label_data, [-1, label_len]), dtype=np.int32)

print(len(label_array))
print(len(test_label_array))

9182
1018


# image, test_image

In [4]:
image_dir = home_dir + image_file
test_image_dir = home_dir + test_image_file

image_list = os.listdir(image_dir)
test_image_list = os.listdir(test_image_dir)

image_list = sorted(image_list, key=lambda x: (int(re.sub('\D','',x)),x))
test_image_list = sorted(test_image_list, key=lambda y: (int(re.sub('\D','',y)),y))

image_array = []
test_image_array = []

with tf.device('/gpu:0') :
    for i in range(len(image_list)) :
        image_list[i] = image_dir + image_list[i]
        image_array.append(np.array(PI.open(image_list[i]).resize((image_resize, image_resize)).convert('L')))

    for j in range(len(test_image_list)) :
        test_image_list[j] = test_image_dir + test_image_list[j]
        test_image_array.append(np.array(PI.open(test_image_list[j]).resize((image_resize, image_resize)).convert('L')))

    image_array = np.reshape(image_array, [-1, image_width*image_height])
    test_image_array = np.reshape(test_image_array, [-1, image_width*image_height])

    image_array = np.array(np.reshape(image_array, [-1, image_width, image_height, 1]), dtype=np.float32)
    test_image_array = np.array(np.reshape(test_image_array, [-1, image_width, image_height, 1]), dtype=np.float32)

print(len(image_array))
print(len(test_image_array))

9182
1018


# feeding value

In [5]:
x = tf.placeholder(dtype=tf.float32, shape=[None, image_width, image_height, 1], name='input_image')
y_ = tf.placeholder(dtype=tf.float32, shape=[None, label_len], name='input_label')
learning_rate = tf.placeholder(dtype=tf.float32, name='learning_rate')

is_training = tf.placeholder(dtype=tf.bool)

# learning layers

In [6]:
def learning_layer(input_data, size, dropout, is_training) :
    layer = tf.layers.conv2d(input_data, size, [3, 3], padding='SAME', activation=tf.math.tanh)
    layer = tf.layers.max_pooling2d(layer, [2, 2], [2, 2], padding='SAME')
    layer = tf.layers.dropout(layer, dropout, is_training)
    
    return layer

with tf.device('/gpu:0') :
    layer0_1 = tf.layers.conv2d(x, 8, [3,3], padding='SAME', activation=tf.math.tanh)
    layer0_2 = tf.layers.conv2d(layer0_1, 128, [3,3], padding='SAME', activation=tf.math.tanh)
    layer1 = learning_layer(input_data=layer0_2, size=256, dropout=0.7, is_training=is_training)
#     layer0_3 = tf.layers.conv2d(layer0_2, 256, [3,3], padding='SAME')
#     layer1 = learning_layer(input_data=layer0_2, size=256, drop_out=0.7, is_training=is_training, name_conv2d="layer1_conv2d", 
#                    name_maxpooling="layer1_maxpooling", name_dropout="layer1_dropout")
    layer1_1 = tf.layers.conv2d(layer1, 128, [3,3], padding='SAME', activation=tf.math.tanh)
    layer1_2 = tf.layers.conv2d(layer1_1, 256, [3,3], padding='SAME', activation=tf.math.tanh)
    layer2 = learning_layer(input_data=layer1_2, size=512, dropout=0.7, is_training=is_training)
    layer2_1 = tf.layers.conv2d(layer2, 256, [3,3], padding='SAME', activation=tf.math.tanh)
    layer2_2 = tf.layers.conv2d(layer2_1, 128, [3,3], padding='SAME', activation=tf.math.tanh)
#     layer2_3 = tf.layers.conv2d(layer2_2, 64, [3,3], padding='SAME')
    layer3 = learning_layer(input_data=layer2_2, size=64, dropout=0.7, is_training=is_training)
#     layer3 = learning_layer(input_data=layer2_2, size=64, drop_out=0.7, is_training=is_training, name_conv2d="layer3_conv2d", 
#                    name_maxpooling="layer3_maxpooling", name_dropout="layer3_dropout")
    layer3_1 = tf.layers.conv2d(layer3, 128, [3,3], padding='SAME', activation=tf.math.tanh)
    layer3_2 = tf.layers.conv2d(layer3_1, 256, [3,3], padding='SAME', activation=tf.math.tanh)
    layer4 = learning_layer(input_data=layer3_2, size=64, dropout=0.7, is_training=is_training)

W1108 10:46:24.632569 18344 deprecation.py:323] From <ipython-input-6-a807bd5638da>:9: conv2d (from tensorflow.python.layers.convolutional) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
W1108 10:46:24.636651 18344 deprecation.py:506] From C:\Users\whdrm\Anaconda3\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W1108 10:46:24.870476 18344 deprecation.py:323] From <ipython-input-6-a807bd5638da>:3: max_pooling2d (from tensorflow.python.layers.pooling) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.MaxPooling2D instead.
W1108 10:46:25.026171 18344 deprecation.py:323] From <ipython-input-6-a807bd5638da>:4: 

In [7]:
fully_layer = tf.contrib.layers.flatten(layer4)
fully_layer = tf.layers.dense(fully_layer, 1024, activation=tf.math.tanh, name='fully_connected_layer')
fully_layer = tf.layers.dropout(fully_layer, 0.5, is_training, name='fully_connected_dropout')

fully_layer2 = tf.layers.dense(fully_layer, 512, activation=tf.math.tanh, name='fully_connected_layer2')
fully_layer2 = tf.layers.dropout(fully_layer2, 0.5, is_training, name='fully_connected_dropout2')

W1108 10:46:27.247069 18344 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

W1108 10:46:27.262667 18344 deprecation.py:323] From C:\Users\whdrm\Anaconda3\lib\site-packages\tensorflow\contrib\layers\python\layers\layers.py:1634: flatten (from tensorflow.python.layers.core) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.flatten instead.
W1108 10:46:27.458343 18344 deprecation.py:323] From <ipython-input-7-b41a9665b332>:2: dense (from tensorflow.python.layers.core) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.dense instead.


In [8]:
logits = tf.layers.dense(fully_layer2, label_len, activation=None)

with tf.name_scope('pred') : 
    pred = tf.nn.softmax(logits)
with tf.name_scope('loss') :
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=y_))
    
with tf.device('/gpu:0') :
    with tf.name_scope('train') :
            train = tf.train.AdamOptimizer(learning_rate).minimize(loss)

correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y_, 1))
acc = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

label_output = tf.argmax(pred, 1)
label_input = tf.argmax(y_, 1)

In [9]:
print(label_output)
print(label_input)

Tensor("ArgMax_2:0", shape=(?,), dtype=int64)
Tensor("ArgMax_3:0", shape=(?,), dtype=int64)


In [10]:
saver = tf.train.Saver()

tf.summary.scalar('scalar_loss', loss)

image_shaped_input = tf.reshape(x, [-1, image_resize, image_resize, 1])
tf.summary.image('input', image_shaped_input)

<tf.Tensor 'input:0' shape=() dtype=string>

In [11]:
config = tf.ConfigProto(log_device_placement = True)
config.gpu_options.allow_growth = True

with tf.Session(config=config) as sess :
    coord = tf.train.Coordinator()
    thread = tf.train.start_queue_runners(sess=sess, coord=coord)
    
#     f = open(log_dir+log_file, 'w')
    
#     saver = tf.train.import_meta_graph('meta_data_name')
#     saver.restore(sess, tf.train.latest_checkpoint('saved'))
    
    sess.run(tf.global_variables_initializer())
    
#     for var in tf.trainable_variables() :
#         tf.summary.histogram(var.name, var)

#     merged_summary = tf.summary.merge_all()  
#     train_writer = tf.summary.FileWriter(merge_dir, sess.graph)
    
    total_batch = int(len(image_list) / batch_size)
    correct_list = []
    incorrect_list = []
    cost_list = []
    cost_sum_list = []
    pred_list = []
    
#     sensitivity_acc = 0
#     specificity_acc = 0
    total_acc = 0
    cost_sum = 0
    cost_flag = 0
    rate_ = 1
    
    start_time = time.time()
    
    print("")
    
    for epoch in range(training_epochs) :
        total_cost = 0
        
        if epoch%50 == 49 :
                rate_ *= 0.8
                
        if epoch%10 == 9 :
#             rate_ *= 0.8
            midpoint = int(time.time() - start_time)
        
            print("")
            print("=======================================================================================")
            print('{:03d}:{:02d}:{:02d}'.format(midpoint//3600, (midpoint%3600//60), midpoint%60))
            print("=======================================================================================")
            print("")
        
        batch_index = np.random.choice(len(image_array), total_batch, replace=False)
                
        for i in range(total_batch) :                
            _, _loss = sess.run([train, loss], feed_dict={x: image_array[[batch_index[i]]], y_: label_array[[batch_index[i]]], 
                                                          is_training: True, learning_rate: initial_rate*rate_})    

#             _, _loss, _merge = sess.run([train, loss, merged_summary], feed_dict={x: image_array[[batch_index[i]]], y_: label_array[[batch_index[i]]],
#                                                                                   is_training: True, learning_rate: initial_rate*rate_})
            
#             saver.save(sess, save_dir, global_step=total_batch, write_meta_graph=False)
            
            total_cost += _loss

        avg_cost = total_cost/total_batch
        print('Epoch : ', '%4d' % (epoch + 1), '    Avg. cost = ', '{:.4f}'.format(avg_cost))            
        
#         train_writer.add_summary(_merge, epoch)
               
    print("")
    print("=======================================================================================")
    print("================================     Training done     ================================")
    print("=======================================================================================")
    print("")
    
    for test in range(len(test_image_array)) :
        _acc, _label_input, _label_output, _pred = sess.run([acc, label_input, label_output, pred], 
                             feed_dict={x: test_image_array[[test]], y_: test_label_array[[test]], is_training: False})

        total_acc += _acc
        pred_list.append(list(_pred[0]))
        
        print("n: ", test, "    label_input: ", _label_input, "    label_output: ", _label_output)
        
        if _label_input == _label_output :
            correct_list.append(test)   
        else :
            incorrect_list.append(test)
        
    print("")
#     print("       SENSITIVITY ACC : ", '{:.5f}'.format(sensitivity_acc / 200))
#     print("       SPECIFICITY ACC : ", '{:.5f}'.format(specificity_acc / 100))
    print("          TOTAL ACC    : ", '{:.5f}'.format(total_acc / len(test_image_array)))
    print("")
    print(pred_list)
    
#     for cnt in range(len(correct_list)) :
#         data = correct_list[cnt]
#         f.write(str(data)+"\n")
#         print(data)
    
#     tf.train.write_graph(sess.graph, './', pb_, as_text=False)
#     tf.train.write_graph(sess.graph, './', txt_, as_text=True)
#     train_writer.add_graph(sess.graph)
    
    end_time = int(time.time() - start_time)
    
    print("")
    print("=======================================================================================")
    print('{:03d}:{:02d}:{:02d}'.format(end_time//3600, (end_time%3600//60), end_time%60))
    print("=======================================================================================")
    print("")
    
#     for cnt_incorrect in range(len(incorrect_list)) :
#         print(incorrect_list[cnt_incorrect])
    
#     f.close()
    
    coord.request_stop()
    coord.join(thread)

W1108 10:46:29.100144 18344 deprecation.py:323] From <ipython-input-11-55e5bb2bde1f>:6: start_queue_runners (from tensorflow.python.training.queue_runner_impl) is deprecated and will be removed in a future version.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
W1108 10:46:29.100144 18344 queue_runner_impl.py:471] `tf.train.start_queue_runners()` was called when no queue runners were defined. You can safely remove the call to this deprecated function.



Epoch :     1     Avg. cost =  0.8416
Epoch :     2     Avg. cost =  0.7151
Epoch :     3     Avg. cost =  0.6591
Epoch :     4     Avg. cost =  0.6137
Epoch :     5     Avg. cost =  0.6546
Epoch :     6     Avg. cost =  0.5853
Epoch :     7     Avg. cost =  0.5499
Epoch :     8     Avg. cost =  0.6190
Epoch :     9     Avg. cost =  0.6269

000:02:53

Epoch :    10     Avg. cost =  0.5984
Epoch :    11     Avg. cost =  0.5965
Epoch :    12     Avg. cost =  0.5026
Epoch :    13     Avg. cost =  0.5251
Epoch :    14     Avg. cost =  0.5598
Epoch :    15     Avg. cost =  0.5010
Epoch :    16     Avg. cost =  0.5191
Epoch :    17     Avg. cost =  0.5562
Epoch :    18     Avg. cost =  0.5777
Epoch :    19     Avg. cost =  0.5363

000:06:03

Epoch :    20     Avg. cost =  0.4970
Epoch :    21     Avg. cost =  0.4489
Epoch :    22     Avg. cost =  0.4624
Epoch :    23     Avg. cost =  0.4645
Epoch :    24     Avg. cost =  0.5474
Epoch :    25     Avg. cost =  0.4485
Epoch :    26     Avg. co

Epoch :   148     Avg. cost =  0.0468
Epoch :   149     Avg. cost =  0.0461

000:48:13

Epoch :   150     Avg. cost =  0.0419
Epoch :   151     Avg. cost =  0.0483
Epoch :   152     Avg. cost =  0.0761
Epoch :   153     Avg. cost =  0.0716
Epoch :   154     Avg. cost =  0.0538
Epoch :   155     Avg. cost =  0.0411
Epoch :   156     Avg. cost =  0.0359
Epoch :   157     Avg. cost =  0.0358
Epoch :   158     Avg. cost =  0.0394
Epoch :   159     Avg. cost =  0.0472

000:51:28

Epoch :   160     Avg. cost =  0.0289
Epoch :   161     Avg. cost =  0.0378
Epoch :   162     Avg. cost =  0.0348
Epoch :   163     Avg. cost =  0.0260
Epoch :   164     Avg. cost =  0.0496
Epoch :   165     Avg. cost =  0.0268
Epoch :   166     Avg. cost =  0.0919
Epoch :   167     Avg. cost =  0.0187
Epoch :   168     Avg. cost =  0.0155
Epoch :   169     Avg. cost =  0.0376

000:54:43

Epoch :   170     Avg. cost =  0.0131
Epoch :   171     Avg. cost =  0.0313
Epoch :   172     Avg. cost =  0.0837
Epoch :   173 

Epoch :   290     Avg. cost =  0.0008
Epoch :   291     Avg. cost =  0.0008
Epoch :   292     Avg. cost =  0.0019
Epoch :   293     Avg. cost =  0.0014
Epoch :   294     Avg. cost =  0.0021
Epoch :   295     Avg. cost =  0.0016
Epoch :   296     Avg. cost =  0.0020
Epoch :   297     Avg. cost =  0.0032
Epoch :   298     Avg. cost =  0.0066
Epoch :   299     Avg. cost =  0.0010

001:37:02

Epoch :   300     Avg. cost =  0.0043
Epoch :   301     Avg. cost =  0.0010
Epoch :   302     Avg. cost =  0.0015
Epoch :   303     Avg. cost =  0.0033
Epoch :   304     Avg. cost =  0.0016
Epoch :   305     Avg. cost =  0.0045
Epoch :   306     Avg. cost =  0.0009
Epoch :   307     Avg. cost =  0.0005
Epoch :   308     Avg. cost =  0.0013
Epoch :   309     Avg. cost =  0.0039

001:40:17

Epoch :   310     Avg. cost =  0.0056
Epoch :   311     Avg. cost =  0.0008
Epoch :   312     Avg. cost =  0.0014
Epoch :   313     Avg. cost =  0.0019
Epoch :   314     Avg. cost =  0.0060
Epoch :   315     Avg. cos

Epoch :   437     Avg. cost =  0.0000
Epoch :   438     Avg. cost =  0.0001
Epoch :   439     Avg. cost =  0.0000

002:22:36

Epoch :   440     Avg. cost =  0.0011
Epoch :   441     Avg. cost =  0.0001
Epoch :   442     Avg. cost =  0.0000
Epoch :   443     Avg. cost =  0.0001
Epoch :   444     Avg. cost =  0.0000
Epoch :   445     Avg. cost =  0.0007
Epoch :   446     Avg. cost =  0.0011
Epoch :   447     Avg. cost =  0.0154
Epoch :   448     Avg. cost =  0.0001
Epoch :   449     Avg. cost =  0.0003

002:25:52

Epoch :   450     Avg. cost =  0.0000
Epoch :   451     Avg. cost =  0.0000
Epoch :   452     Avg. cost =  0.0000
Epoch :   453     Avg. cost =  0.0000
Epoch :   454     Avg. cost =  0.0000
Epoch :   455     Avg. cost =  0.0000
Epoch :   456     Avg. cost =  0.0000
Epoch :   457     Avg. cost =  0.0000
Epoch :   458     Avg. cost =  0.0001
Epoch :   459     Avg. cost =  0.0001

002:29:07

Epoch :   460     Avg. cost =  0.0002
Epoch :   461     Avg. cost =  0.0000
Epoch :   462 

...

n:  253     label_input:  [0]     label_output:  [1]
n:  254     label_input:  [0]     label_output:  [0]
n:  255     label_input:  [0]     label_output:  [0]
n:  256     label_input:  [0]     label_output:  [1]
n:  257     label_input:  [0]     label_output:  [0]
n:  258     label_input:  [0]     label_output:  [0]
n:  259     label_input:  [0]     label_output:  [0]
n:  260     label_input:  [0]     label_output:  [0]
n:  261     label_input:  [0]     label_output:  [0]
n:  262     label_input:  [0]     label_output:  [0]
n:  263     label_input:  [0]     label_output:  [0]
n:  264     label_input:  [0]     label_output:  [1]
n:  265     label_input:  [0]     label_output:  [0]
n:  266     label_input:  [0]     label_output:  [0]
n:  267     label_input:  [0]     label_output:  [0]
n:  268     label_input:  [0]     label_output:  [0]
n:  269     label_input:  [0]     label_output:  [0]
n:  270     label_input:  [0]     label_output:  [1]
n:  271     label_input:  [0]     label_output

n:  417     label_input:  [1]     label_output:  [1]
n:  418     label_input:  [1]     label_output:  [1]
n:  419     label_input:  [1]     label_output:  [1]
n:  420     label_input:  [1]     label_output:  [2]
n:  421     label_input:  [1]     label_output:  [2]
n:  422     label_input:  [1]     label_output:  [1]
n:  423     label_input:  [1]     label_output:  [1]
n:  424     label_input:  [1]     label_output:  [1]
n:  425     label_input:  [1]     label_output:  [2]
n:  426     label_input:  [1]     label_output:  [1]
n:  427     label_input:  [1]     label_output:  [1]
n:  428     label_input:  [1]     label_output:  [1]
n:  429     label_input:  [1]     label_output:  [1]
n:  430     label_input:  [1]     label_output:  [1]
n:  431     label_input:  [1]     label_output:  [1]
n:  432     label_input:  [1]     label_output:  [1]
n:  433     label_input:  [1]     label_output:  [1]
n:  434     label_input:  [1]     label_output:  [1]
n:  435     label_input:  [1]     label_output

n:  581     label_input:  [2]     label_output:  [2]
n:  582     label_input:  [2]     label_output:  [2]
n:  583     label_input:  [2]     label_output:  [2]
n:  584     label_input:  [2]     label_output:  [2]
n:  585     label_input:  [2]     label_output:  [2]
n:  586     label_input:  [2]     label_output:  [2]
n:  587     label_input:  [2]     label_output:  [2]
n:  588     label_input:  [2]     label_output:  [2]
n:  589     label_input:  [2]     label_output:  [2]
n:  590     label_input:  [2]     label_output:  [2]
n:  591     label_input:  [2]     label_output:  [2]
n:  592     label_input:  [2]     label_output:  [2]
n:  593     label_input:  [2]     label_output:  [2]
n:  594     label_input:  [2]     label_output:  [2]
n:  595     label_input:  [2]     label_output:  [2]
n:  596     label_input:  [2]     label_output:  [2]
n:  597     label_input:  [2]     label_output:  [2]
n:  598     label_input:  [2]     label_output:  [2]
n:  599     label_input:  [2]     label_output

n:  746     label_input:  [2]     label_output:  [2]
n:  747     label_input:  [2]     label_output:  [2]
n:  748     label_input:  [2]     label_output:  [2]
n:  749     label_input:  [2]     label_output:  [2]
n:  750     label_input:  [2]     label_output:  [2]
n:  751     label_input:  [2]     label_output:  [2]
n:  752     label_input:  [2]     label_output:  [2]
n:  753     label_input:  [2]     label_output:  [2]
n:  754     label_input:  [2]     label_output:  [2]
n:  755     label_input:  [2]     label_output:  [2]
n:  756     label_input:  [2]     label_output:  [2]
n:  757     label_input:  [2]     label_output:  [2]
n:  758     label_input:  [2]     label_output:  [2]
n:  759     label_input:  [2]     label_output:  [2]
n:  760     label_input:  [2]     label_output:  [2]
n:  761     label_input:  [2]     label_output:  [2]
n:  762     label_input:  [2]     label_output:  [2]
n:  763     label_input:  [2]     label_output:  [2]
n:  764     label_input:  [2]     label_output

n:  908     label_input:  [2]     label_output:  [2]
n:  909     label_input:  [2]     label_output:  [2]
n:  910     label_input:  [2]     label_output:  [2]
n:  911     label_input:  [2]     label_output:  [2]
n:  912     label_input:  [2]     label_output:  [0]
n:  913     label_input:  [2]     label_output:  [2]
n:  914     label_input:  [2]     label_output:  [2]
n:  915     label_input:  [2]     label_output:  [2]
n:  916     label_input:  [2]     label_output:  [2]
n:  917     label_input:  [2]     label_output:  [2]
n:  918     label_input:  [2]     label_output:  [2]
n:  919     label_input:  [2]     label_output:  [2]
n:  920     label_input:  [2]     label_output:  [2]
n:  921     label_input:  [2]     label_output:  [0]
n:  922     label_input:  [2]     label_output:  [2]
n:  923     label_input:  [2]     label_output:  [0]
n:  924     label_input:  [2]     label_output:  [2]
n:  925     label_input:  [2]     label_output:  [2]
n:  926     label_input:  [2]     label_output